In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc import io
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import RefPort

from lava.lib.dl import netx

from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")

In [2]:
net = netx.hdf5.Network(net_config='net_snn.net')
print(net)

|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Dense     |    1|    1|   64|     |     |     |     |     |False|
|Dense     |    1|    1|  128|     |     |     |     |     |False|
|Dense     |    1|    1|    2|     |     |     |     |     |False|


In [3]:
print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.block:5s} : {l.name:10s}, shape : {l.shape}')

There are 3 layers in network:
Dense : Process_1 , shape : (64,)
Dense : Process_4 , shape : (128,)
Dense : Process_7 , shape : (2,)


In [4]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [0]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

In [5]:
nb_inputs  = 2
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 32

import scipy.io

from dataset import Dataset, SpikingDataset, ARegDataset, RegSpikingDataset, RegTorchSpikingDataset
train_data = RegTorchSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
if np.isscalar(train_data[0][1]):
    print("True")
print(len(train_data))
print(train_data[0][1].shape)
print(train_data[0][0].shape)

7521
(2,)
(8, 100)


In [39]:
# Import Process level primitives
import typing as ty 
from typing import Iterable, Tuple, Union

from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort

# Import parent classes for ProcessModels
from lava.magma.core.model.sub.model import AbstractSubProcessModel
from lava.magma.core.model.py.model import PyLoihiProcessModel

# Import ProcessModel ports, data-types
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType

# Import execution protocol and hardware resources
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.resources import CPU

# Import decorators
from lava.magma.core.decorator import implements, requires, tag


class SpikeInput(AbstractProcess):
    """Reads image data from the MNIST dataset and converts it to spikes.
    The resulting spike rate is proportional to the pixel value."""

    def __init__(self,
                 dataset: Iterable,
                 sequence_length : ty.Optional[int] = 2,
                 num_input_node : ty.Optional[int] = 4,
                 num_steps: ty.Optional[int] = 100):
        super().__init__()
        self.sequence_length = Var(shape=(1,), init=sequence_length)
        self.num_input_node = Var(shape=(1,), init=num_input_node) 
        shape = (sequence_length * num_input_node,)
        self.spikes_out = OutPort(shape=shape)
        self.num_steps = Var(shape=(1,), init=num_steps)
        self.input_data = Var(shape=shape)
        self.proc_params['saved_dataset'] = dataset
        
@implements(proc=SpikeInput, protocol=LoihiProtocol)
@requires(CPU)
class PySpikeInputModel(PyLoihiProcessModel):
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    input_data: np.ndarray = LavaPyType(np.ndarray, bool, precision=1)
    num_steps: int = LavaPyType(int, int, precision=32)
    sequence_length: int = LavaPyType(int, int, precision=32)
    num_input_node: int = LavaPyType(int, int, precision=32)
    
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.dataset = self.proc_params['saved_dataset']
        self.curr_id = 0
        self.sample_time = 0

    def post_guard(self):
        """Guard function for PostManagement phase.
        """
        if self.sample_time == self.num_steps:
            return True
        return False
        #return True

    def run_post_mgmt(self):
        """Post-Management phase: executed only when guard function above 
        returns True.
        """
        self.curr_id += 1
        self.sample_time = 0
    
    def run_spk(self):
        """Spiking phase: executed unconditionally at every time-step
        """
        output = self.dataset[self.curr_id][0][:, self.sample_time]
        print(self.sample_time, self.curr_id, output)
        self.spikes_out.send(output)
        self.sample_time += 1
        #print(self.time_step)
        #print(self.input_data)
        #print(self.dataset[self.curr_id][(self.time_step - 1) % (self.num_steps)])
        
@implements(proc=SpikeInput, protocol=LoihiProtocol)
@tag('fixed_pt')
class PySpikeModelFixed(PySpikeInputModel):
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, np.int32)
    input_data: np.ndarray = LavaPyType(np.ndarray, np.int32)


@implements(proc=SpikeInput, protocol=LoihiProtocol)
@tag('floating_pt')
class PySpikeModelFloat(PySpikeInputModel):
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)
    input_data: np.ndarray = LavaPyType(np.ndarray, float)

In [44]:
from dataset import Dataset, SpikingDataset, ARegDataset, RegSpikingDataset, RegTorchSpikingDataset
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense  

train_data = RegTorchSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
print(len(train_data))
print(train_data[0][1].shape)
print(train_data[0][0].shape)
sequence_length = 2
num_input_node = 4

spike_input = SpikeInput(
                         dataset = train_data,
                         sequence_length = sequence_length,
                         num_input_node = num_input_node,
                         num_steps = nb_steps)


dense = Dense(weights=125 * np.ones((8, 8)))
spike_input.spikes_out.connect(dense.s_in)

7521
(2,)
(8, 3)


In [45]:
for idx in range(2):
    data = train_data[idx][0]
    for j in range(data.shape[1]):
        print(data[:, j])
        print(j)

[0 0 0 0 1 1 0 1]
0
[0 0 0 0 1 1 0 1]
1
[0 0 0 0 1 0 0 1]
2
[1 1 0 1 0 1 0 1]
0
[0 1 0 1 0 1 0 0]
1
[1 1 0 1 0 1 0 0]
2


In [46]:
for idx in range(2):
    print("cur_idx", idx)
    dense.run(
        condition=RunSteps(num_steps=nb_steps),
        run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                             select_tag='fixed_pt'))
    x = spike_input.input_data.get().astype(bool)
    
dense.stop()

cur_idx 0
0 0 [0 0 0 0 1 1 0 1]
1 0 [0 0 0 0 1 1 0 1]
2 0 [0 0 0 0 1 1 0 1]
0 1 [1 0 0 1 0 1 0 0]
1 1 [0 1 0 1 0 1 0 1]
2 1 [1 1 0 1 0 1 0 1]
cur_idx 1


In [6]:
from lava.proc import io

# Instantiate the processes
dataloader = io.dataloader.SpikeDataloader(dataset=train_data, offset=100)
output_logger = io.sink.RingBuffer(shape=net.out_layer.shape, buffer=nb_steps)
#gt_logger = io.sink.RingBuffer(shape=(2,), buffer=nb_steps)


# Connect the input to the network:
#dataloader.ground_truth.connect(gt_logger.a_in)
dataloader.s_out.connect(net.inp)

# Connect network-output to the output process
net.out.connect(output_logger.a_in)

In [7]:
pretrained_model = torch.load('./net_dnn.pth', map_location=torch.device('cpu'))
class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, output_size)
        #self.fc2 = nn.Linear(64, 128)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x):
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        #x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc3(x)
        #x = self.act(x)
        #x = self.fc4(x)
        return x
    
net_dnn = DNNNetwork(2 * 100, 2).to(device)
net_dnn.load_state_dict(pretrained_model)

<All keys matched successfully>

In [49]:
class CustomRunConfig(Loihi1SimCfg):
    def select(self, proc, proc_models):
        # customize run config to always use float model for io.sink.RingBuffer
        if isinstance(proc, io.sink.RingBuffer):
            return io.sink.PyReceiveModelFloat
        else:
            return super().select(proc, proc_models)

In [8]:

for i in range(20):
    run_condition = RunSteps(num_steps=nb_steps)
    run_config = Loihi1SimCfg(select_tag='fixed_pt', select_sub_proc_model=True)
    net.run(condition=run_condition, run_cfg=run_config) 
    results = output_logger.data.get()
    results = torch.Tensor(results).to(device)
    results = torch.unsqueeze(results, dim=0)
    print(type(results), results.shape)
    results = net_dnn(results)
    
    dataloader.data.get().astype(bool)
    #dataloader.s_out.get().astype(bool)
    print(results)
#gts = gt_logger.data.get()
net.stop()

<class 'torch.Tensor'> torch.Size([1, 2, 100])


AttributeError: 'OutPort' object has no attribute 'get'

In [29]:
print(results)

tensor([[-0.0148, -0.1659]], grad_fn=<AddmmBackward0>)
